<a href="https://colab.research.google.com/github/GretaDanesi18/Olive_Detection_Tracking/blob/main/target_detector/YOLOv5_TDet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SETUP**

Per prima cosa cloniamo la repo contente YOLOv5 e installiamo i requisiti

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Successivamente importiamo le librerie necessarie

In [ ]:
import torch 
import os
import random 
import shutil
import glob
from IPython.display import Image,display
from PIL import Image, ImageDraw
import cv2
import pandas as pd
import subprocess
from numpy import source


Controlliamo l'utilizzo del device GPU oppure CPU

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)

##**CREAZIONE DATASET TARGET DETECTOR**

Caricare la cartella contenente le pseudo labels("Results__PseudoLabels") generate tramite i software contenuti nella seguente repo:
https://github.com/Lio320/GraphTracking.git

Possiamo caricarle in diversi modi:


* in locale 
```
from google.colab import files
uploaded = files.upload()
```
*  con google drive
```
from google.colab import drive
drive.mount('/content/gdrive')
```
* con github
```
!git clone "inserire url della repo in cui si trovano le pseudo labels"
```










In [ ]:
!git clone https://github.com/GretaDanesi18/Olive_Detection_Tracking.git

Creiamo le cartellle per il dataset del target detector.

La struttura del dataset del target detector è la stessa del source detector con l'aggiunta delle cartella per il test




```
/path/to/dataset/...

+--images
|   +---train
         +-----image_1.jpg
         +-----image_2.jpg
         +-----image_n.jpg
|   +---val
        +-----image_1.jpg
        +-----image_2.jpg
        +-----image_n.jpg
|   +---test
        +-----image_1.jpg
        +-----image_2.jpg
        +-----image_n.jpg
+--labels
|   +---train
         +-----label_1.txt
         +-----label_2.txt
         +-----label_n.txt
|   +---val
        +-----label_1.txt
        +-----label_2.txt
        +-----label_n.txt
|   +---test
        +-----label_1.txt
        +-----label_2.txt
        +-----label_n.txt

```


In [ ]:
try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/images/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/images/train')

except OSError:
    print('Errore nella creazione della cartella Pseudo_Labels/images/train')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/images/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/images/val')

except OSError:
    print('Errore nella creazione della cartella Pseudo_Labels/images/val')


try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/images/test'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/images/test')

except OSError:
    print('Errore nella creazione della cartella Pseudo_Labels/images/test')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/labels/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/labels/train')

except OSError:
    print('Errore nella creazione della cartella Pseudo_Labels/labels/train')


try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/labels/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/labels/val')

except OSError:
    print('Errore nella creazione della cartella Pseudo_Labels/labels/val')


try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/labels/test'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/labels/test')

except OSError:
   print('Errore nella creazione della cartella Pseudo_Labels/labels/test')



Faccio lo split del target dataset cosi da ottenere il dataset per il training e per la validation

In [ ]:
#array che contiene i file
image = []
label = [] 

#path
all_data_path = '/content/yolov5/Olive_Detection_Tracking/target_detector/pseudo_labels_dataset'
train_image_path = '/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/images/train'
val_image_path = '/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/images/val'
test_image_path = '/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/images/test'
train_label_path ='/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/labels/train'
val_label_path = '/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/labels/val'
test_label_path = '/content/yolov5/Olive_Detection_Tracking/Pseudo_Labels/labels/test'

#inserisco images nella cartella per il test
source_image = '/content/yolov5/Olive_Detection_Tracking/target_detector/test_dataset/images'
destination_image =test_image_path
lista_image = os.listdir(source_image)
for frame in lista_images:
    name = frame.split('.')[0]
    source = source_image+'/'+frame
    destination = destination_image+'/'+frame
    shutil.copyfile(source,destination)

#inserisco labels nella cartella per il test
source_label = '/content/yolov5/Olive_Detection_Tracking/target_detector/test_dataset/labels'
destination_label =test_label_path
lista_label = os.listdir(source_label)
for label in lista_label:
    name = label.split('.')[0]
    source = source_label+'/'+label
    destination = destination_label+'/'+label
    shutil.copyfile(source,destination)


#percentuale per train, val
val_ratio = 0.2
train_ratio = 0.6

#inserisco file negli array corrispondenti
for (dirname, dirs, files) in os.walk(all_data_path):
    for filename in files:
        if filename.endswith('.txt'):
            label.append(filename)
        else:
            image.append(filename)

#numero di immagini per train e per val
num_train = int(len(image)*train_ratio)
num_val = int(len(image)*val_ratio)
num_test = int(len(lista_image))

print("Immagini per training: ", num_train)
print("Immagini per validation: ", num_val)
print("Immagini per test: ", num_test)

In [ ]:
#inserisco image e label in train
for i in range(num_train):
   file_image = random.choice(image)
   file_image_name = file_image.split('.')[0]
   file_label =  file_image_name+'.txt'

   #sposto i file nella cartella train
   shutil.copy(os.path.join(all_data_path, file_image), os.path.join(train_image_path, file_image))
   shutil.copy(os.path.join(all_data_path, file_label), os.path.join(train_label_path, file_label))

   #rimuovo i file da arrays
   image.remove(file_image)
   label.remove(file_label)
    

#inserisco image e label in val  
for i in range(num_val):
    file_image =random.choice(image) # get name of random image from origin dir
    file_image_name = file_image.split('.')[0]
    file_label = file_image_name+'.txt'

    #move both files into train dir
    shutil.copy(os.path.join(all_data_path, file_image), os.path.join(val_image_path, file_image))
    shutil.copy(os.path.join(all_data_path, file_label), os.path.join(val_label_path, file_label))
    
    #rimuovo i file da arrays
    image.remove(file_image)
    label.remove(file_label)

Cambiare i path all' interno del file target_dataset.yaml in accordo con il proprio dataset

In [ ]:
#inserisco target_dataset.yal nella directory 'data'
source = 'Olive_Detection_Tracking/target_detector/target_dataset.yaml'
destination = 'data/target_dataset.yaml'
shutil.copy(source,destination)

## **ADDESTRAMENTO TARGET DETECTOR**

Usiamo Weights & Biases per mostrare i risultati.

Eventualmente possiamo usare anche tensorboard lanciando il seguente codice dopo aver avviato il training.

```
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs
```



In [ ]:
#Weights & Biases
%pip install -q wandb
import wandb
wandb.login()

Avviamo l'addestramento

In [ ]:
!python train.py --img 416 --batch 16 --epochs 30 --data target_dataset.yaml --weights yolov5s.pt --cache 

## **VALIDATION TARGET DETECTOR**

Dopo aver fatto il training facciamo la validation sul dateset del test


In [ ]:
!python val.py --img 416 --batch 16 --data target_dataset.yaml --weights runs/train/exp3/weights/best.pt --task test --save-txt 

In [ ]:

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/val/exp5/*.jpg'): #assuming JPG
    display(Image(filename=imageName,width=1280))
    print("\n")


In [ ]:
for imageName in glob.glob('/content/yolov5/runs/val/exp6/*.png'): #assuming JPG
    display(Image(filename=imageName,width=960))
    print("\n")

## **INFERENZA TARGET DETECTOR**

Infine faccio l'inferenza su un nuovo set di immagini creato estraendo i frame dal video2 che si trova al seguente path Olive_Detection_Tracking_dataset_video/videos 

In [ ]:
!python detect.py --source Olive_Detection_Tracking/dataset_video/frame_video/frame_video2 --data data/target_dataset.yaml --weights runs/train/exp/weights/best.pt --save-txt --conf-thres 0.40